# Instalar Bibliotecas

In [1]:
#!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached psycopg2-2.9.9-cp312-cp312-win_amd64.whl.metadata (4.5 kB)
Using cached psycopg2-2.9.9-cp312-cp312-win_amd64.whl (1.2 MB)



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\Rodrigo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Importar Bibliotecas

In [1]:
import sidrapy
import requests, json
import pyarrow as pa
import pyarrow.parquet as pq
import ibge.localidades as api_ibge
import pandas as pd

# VARIAVEIS GLOBAIS

In [2]:
PATH = "C:\\PROJETOS\\ibge_integration\\RAW_DATA\\" # SUBSTITUA UM PATH PARA UM DE SUA PREFERENCIA OU UM BUCKET
PATH = "E:\\AMBIENTE DEV\\PROJETOS\\ibge_integration\\RAW_DATA\\" # SUBSTITUA UM PATH PARA UM DE SUA PREFERENCIA OU UM BUCKET


# Funções

In [3]:
def json_to_parquet(json_data, parquet_filename):
    """
    Converte dados em formato JSON para o formato Parquet.

    Parameters:
    - json_data (list): Lista contendo dados no formato JSON.
    - parquet_filename (str): Caminho para o arquivo Parquet de destino.

    Returns:
    - bool: True se a conversão for bem-sucedida, False em caso de erro.

    Raises:
    - Exception: Erro ao salvar ou ler o arquivo Parquet.

    Example:
    >>> json_data = [{"col1": 1, "col2": "A"}, {"col1": 2, "col2": "B"}]
    >>> parquet_filename = "output.parquet"
    >>> json_to_parquet(json_data, parquet_filename)
    True
    """

    try:
        # Formatação do JSON para estrutura de dicionário
        json_formated = {chave: [d[chave] for d in json_data] for chave in json_data[0].keys()}
       
        # Criar uma tabela PyArrow a partir do dicionário
        table = pa.Table.from_pydict(json_formated)
        
        # Escrever a tabela no arquivo Parquet
        with pq.ParquetWriter(parquet_filename, schema=table.schema) as writer:
            writer.write_table(table)

        print(f"Arquivo Parquet '{parquet_filename}' gerado com sucesso.")

        # Validar se os dados foram salvos corretamente lendo o arquivo Parquet
        tabela_lida = pq.read_table(parquet_filename)
        
        return True

    except Exception as e:
        print(f"Erro ao salvar ou ler o arquivo Parquet: {e}")
        return False


# Estados e Regiões

In [4]:



# Lendo dados de regioes pelo ibge e jogando no lake (RAW_DATA)
json_region = api_ibge.Regioes().json()
json_to_parquet(json_data=json_region,parquet_filename=f'{PATH}\\regioes.parquet')

# Lendo dados de estado pelo ibge e jogando no lake (RAW_DATA)
json_state = api_ibge.Estados().json()
json_to_parquet(json_data=json_state,parquet_filename=f'{PATH}\\estados.parquet')

# Lendo dados de municipio pelo ibge e jogando no lake (RAW_DATA)
json_substate = api_ibge.Municipios().getDados()
json_to_parquet(json_data=json_substate,parquet_filename=f'{PATH}\\municipios.parquet')




Arquivo Parquet 'E:\AMBIENTE DEV\PROJETOS\ibge_integration\RAW_DATA\\regioes.parquet' gerado com sucesso.
Arquivo Parquet 'E:\AMBIENTE DEV\PROJETOS\ibge_integration\RAW_DATA\\estados.parquet' gerado com sucesso.
Arquivo Parquet 'E:\AMBIENTE DEV\PROJETOS\ibge_integration\RAW_DATA\\municipios.parquet' gerado com sucesso.


True

# IDH

In [5]:
url_idh = "https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/{state_id}"

"""
URL do IBGE para extrair os resultados da Pesquisa de IDH (código 37).

Parameters:
- state_id (str): ID do estado para o qual os resultados serão extraídos.

Returns:
- str: URL formatada para uma solicitação específica de um estado.
"""

# Percorre os dados de idh
for state in json_state:
    result = requests.get(url_idh.format(state_id=state['id']))
    
    if result.status_code == 200:
        json_idh = json.loads(result.text)
        
        for search in json_idh:
            data_list = []
            cd_search = search['id']
            
            for res in search['res']:
                res.pop('notas', None)
                data = [{'cd_pesquisa': cd_search, 'cd_estado': state['id'], 'ano': ano, 'valor': valor} for ano, valor in res['res'].items()]
                
                # Exibe a lista de dicionários
                for entry in data:
                    data_list.append(entry)

            string_json = json.dumps(data_list, ensure_ascii=False)
            
            rjson = json.loads(string_json)
            print(rjson)
            
            json_to_parquet(json_data=rjson, parquet_filename=f"{PATH}\\idh\\pesquisa37_30255_uf_{state['id']}.parquet")


[{'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '1991', 'valor': '0.407'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2000', 'valor': '0.537'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2010', 'valor': '0.690'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2012', 'valor': '0.694'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2013', 'valor': '0.687'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2014', 'valor': '0.692'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2015', 'valor': '0.706'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2016', 'valor': '0.715'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2017', 'valor': '0.721'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2018', 'valor': '0.73'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2019', 'valor': '0.73'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2020', 'valor': '0.739'}, {'cd_pesquisa': 30255, 'cd_estado': 11, 'ano': '2021', 'valor': '0.7'}]
Arquivo Parquet 'E:\AMBIENTE DEV\PROJETOS

# POPULACIONAL

In [6]:
periodos = ['2018', '2019', '2020', '2021']
url_population = 'https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/{periodo}/variaveis/9324?localidades=N6[N3[{state_id}]]'

"""
URL do IBGE para extrair os resultados da Pesquisa de População (código 6579, variável 9324).

Parameters:
- periodo (str): Período da pesquisa para o qual os resultados serão extraídos.
- state_id (str): ID do estado para o qual os resultados serão extraídos.

Returns:
- str: URL formatada para uma solicitação específica de um estado e período.
"""

for periodo in periodos:
    for state in json_state:
        new_url = url_population.format(state_id=state['id'], periodo=periodo)
        print(new_url)
        result = requests.get(new_url)

        # Verifique se a chamada à API foi bem-sucedida
        if result.status_code == 200:
            # Carregue os dados da API em um dicionário
            json_population = json.loads(result.text)
            # percorre o json
            for search in json_population:
                data_list = []
                cd_search = f"{search['id']}_6579"

                for res in search['resultados']:
                    for series in res['series']:
                        data = [{'cd_pesquisa': cd_search, 'municipio': series['localidade']['id'], 'ano': ano, 'valor': valor} for ano, valor in series['serie'].items()]

                        # Exiba a lista de dicionários
                        for entry in data:
                            data_list.append(entry)

                string_json = json.dumps(data_list, ensure_ascii=False)
                rjson = json.loads(string_json)

            print(data_list)

            json_to_parquet(json_data=rjson, parquet_filename=f"{PATH}\\populacao\\pesquisa6579_9324_uf_{state['id']}_ano_{periodo}.parquet")


https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2018/variaveis/9324?localidades=N6[N3[11]]
[{'cd_pesquisa': '9324_6579', 'municipio': '1100015', 'ano': '2018', 'valor': '23167'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100023', 'ano': '2018', 'valor': '106168'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100031', 'ano': '2018', 'valor': '5438'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100049', 'ano': '2018', 'valor': '84813'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100056', 'ano': '2018', 'valor': '16444'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100064', 'ano': '2018', 'valor': '16227'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100072', 'ano': '2018', 'valor': '7567'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100080', 'ano': '2018', 'valor': '17855'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100098', 'ano': '2018', 'valor': '32047'}, {'cd_pesquisa': '9324_6579', 'municipio': '1100106', 'ano': '2018', 'valor': '45783'}, {'cd_pesquisa': '9324_6

# PIB

In [7]:
periodos = ['2018', '2019', '2020', '2021']
url_pib = 'https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/{periodo}/variaveis/37?localidades=N6[N3[{state_id}]]'

"""
URL do IBGE para extrair os resultados da Pesquisa de PIB (código 5938, variável 37).

Parameters:
- periodo (str): Período da pesquisa para o qual os resultados serão extraídos.
- state_id (str): ID do estado para o qual os resultados serão extraídos.

Returns:
- str: URL formatada para uma solicitação específica de um estado e período.
"""

for periodo in periodos:
    # A IDEIA É EXTRAIR sempre d-1 mas como não tem essa funcionalidade estou extraindo por estado
    for state in json_state:
        
        new_url = url_pib.format(state_id=state['id'], periodo=periodo)
        print(new_url)
        result = requests.get(new_url)

        # Verifique se a chamada à API foi bem-sucedida
        if result.status_code == 200:
            # Carregue os dados da API em um dicionário
            json_pib = json.loads(result.text)

            # percorre o json
            for search in json_pib:
                data_list = []
                cd_search = f"{search['id']}_5938"

                for res in search['resultados']:
                    for series in res['series']:
                        data = [{'cd_pesquisa': cd_search, 'municipio': series['localidade']['id'], 'ano': ano, 'valor': valor} for ano, valor in series['serie'].items()]

                        # Exiba a lista de dicionários
                        for entry in data:
                            data_list.append(entry)
                        
                string_json = json.dumps(data_list, ensure_ascii=False)
                rjson = json.loads(string_json)

        print(data_list)

        json_to_parquet(json_data=rjson, parquet_filename=f"{PATH}\\pib\\pesquisa5938_37_uf_{state['id']}_ano_{periodo}.parquet")


https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/2018/variaveis/37?localidades=N6[N3[11]]
[{'cd_pesquisa': '37_5938', 'municipio': '1100015', 'ano': '2018', 'valor': '498980'}, {'cd_pesquisa': '37_5938', 'municipio': '1100023', 'ano': '2018', 'valor': '2464704'}, {'cd_pesquisa': '37_5938', 'municipio': '1100031', 'ano': '2018', 'valor': '140503'}, {'cd_pesquisa': '37_5938', 'municipio': '1100049', 'ano': '2018', 'valor': '2175840'}, {'cd_pesquisa': '37_5938', 'municipio': '1100056', 'ano': '2018', 'valor': '470647'}, {'cd_pesquisa': '37_5938', 'municipio': '1100064', 'ano': '2018', 'valor': '330232'}, {'cd_pesquisa': '37_5938', 'municipio': '1100072', 'ano': '2018', 'valor': '320416'}, {'cd_pesquisa': '37_5938', 'municipio': '1100080', 'ano': '2018', 'valor': '230151'}, {'cd_pesquisa': '37_5938', 'municipio': '1100098', 'ano': '2018', 'valor': '606072'}, {'cd_pesquisa': '37_5938', 'municipio': '1100106', 'ano': '2018', 'valor': '837459'}, {'cd_pesquisa': '37_5938', 'muni